# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [13]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [11]:
city_weather_df = pd.read_csv("output_data/cities.csv")
# city_weather.head()
city_weather_data_df = city_weather_df.drop(columns = 'Unnamed: 0')
city_weather_data_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Vila Velha,-20.3297,-40.2925,64.35,94,40,3.44,BR,1624169083
1,Mataura,-46.1927,168.8643,45.48,95,76,3.51,NZ,1624168909
2,Cap Malheureux,-19.9842,57.6142,75.47,59,38,11.99,MU,1624169083
3,Wu’an,31.6836,112.0044,73.04,62,100,2.64,CN,1624169084
4,Rikitea,-23.1203,-134.9692,75.20,77,10,19.93,PF,1624169084
...,...,...,...,...,...,...,...,...,...
574,Taquara,-29.6506,-50.7806,58.78,89,100,1.39,BR,1624169243
575,Kaeo,-35.1000,173.7833,58.32,93,33,15.59,NZ,1624169243
576,Evensk,61.9500,159.2333,53.17,63,78,2.64,RU,1624169243
577,Yebaishou,41.3975,119.6408,81.25,26,59,15.48,CN,1624169244


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [16]:
# Configure gmaps with API key
gmaps.configure(api_key=g_key)

# Lat and Lng as locations
locations = city_weather_data_df[["Lat", "Lng"]]

# Add humidity as the weight

humid_weight = city_weather_data_df["Humidity"]


In [25]:
# Create a humidity Heatmap layer

fig = gmaps.figure(center=(0.0, 0.0), zoom_level=2)

heat_layer = gmaps.heatmap_layer(locations, weights=humid_weight, dissipating=False, max_intensity=100,
                                point_radius = 2)

fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [27]:
# Narrow down the cities to fit weather conditions: max temp between 70 and 80, wind speed less than 10 mph,
# and cloudiness equals to 0. 
narrowed_city_df = city_weather_data_df.loc[(city_weather_data_df["Wind Speed"] <= 10) & 
                                            (city_weather_data_df["Cloudiness"] == 0) &
                                            (city_weather_data_df["Max Temp"] >= 70) & 
                                            (city_weather_data_df["Max Temp"] <= 80)].dropna()

narrowed_city_df

SyntaxError: invalid syntax (<ipython-input-27-97fdcd2a4a67>, line 4)

### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
